# LinkedIn Jobs Scraper Việt Nam (IT) - Phiên bản Jupyter Notebook

Script này scrape job IT từ LinkedIn Việt Nam sử dụng Voyager API unofficial.

**Cảnh báo**:
- Đây là unofficial API → LinkedIn có thể block IP/account bất cứ lúc nào.
- Phải cập nhật `csrf-token` và `li_at` cookie thường xuyên (lấy từ DevTools khi login LinkedIn).
- Chạy chậm, có sleep để tránh block.
- Năm 2026: Endpoint có thể thay đổi → kiểm tra Network tab trên browser nếu lỗi.

Chạy từng cell theo thứ tự.

In [17]:
# Cell 1: Import thư viện
import requests
import json
from urllib.parse import quote
import csv
import time
import datetime
import os
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor, as_completed

print("Import thành công!")

Import thành công!


In [18]:
# Cell 2: Cấu hình chính (CHỈNH Ở ĐÂY)

SUMMARY_FILE = "linkedin_job_summaries.jsonl"       # File lưu job đã xử lý thành công
CSV_FILE = "linkedin_jobs_vietnam_it_full.csv"     # File CSV output
COUNT = 100                                        # Số job mỗi trang (max ~100)
MAX_WORKERS = 8                                    # Số luồng song song lấy detail
RETRY_COUNT = 3                                    # Số lần retry khi fail detail

load_dotenv(); 

# Headers & Cookies - BẮT BUỘC CẬP NHẬT THƯỜNG XUYÊN!

headers = {
    "accept": os.getenv("LI_ACCEPT"),
    "csrf-token": os.getenv("LI_CSRF_TOKEN"),
    "referer": os.getenv("LI_REFERER"),
    "user-agent": os.getenv("LI_USER_AGENT"),
    "x-li-lang": os.getenv("LI_LANG"),
    "x-restli-protocol-version": os.getenv("LI_RESTLI_VERSION"),
}

cookies = {
    "li_at": os.getenv("LI_AT"),
    "JSESSIONID": f'"{os.getenv("JSESSIONID")}"',  # LinkedIn BẮT BUỘC có dấu "
}

session = requests.Session()
session.headers.update(headers)
session.cookies.update(cookies)

print("Cấu hình hoàn tất.")

Cấu hình hoàn tất.


In [19]:
# Cell 3: Load keywords từ file (hoặc mặc định)
def load_keywords(filename="keywords.txt"):
    try:
        with open(filename, "r", encoding="utf-8") as f:
            keywords = [line.strip() for line in f if line.strip()]
        print(f"Đã tải {len(keywords)} từ khóa: {keywords}")
        return keywords
    except FileNotFoundError:
        print("Không tìm thấy 'keywords.txt' → dùng mặc định")
        return ["it", "software engineer", "data analyst", "developer vietnam"]

keywords_list = load_keywords()
print("\nDanh sách keywords:", keywords_list)

Đã tải 229 từ khóa: ['IT', 'Information Technology', 'Công nghệ thông tin', 'CNTT', 'Computer Science', 'CS', 'Software', 'Hardware', 'Firmware', 'System', 'Application', 'Platform', 'Infrastructure', 'Technology', 'Tech', 'Lập trình', 'Lập trình viên', 'Programmer', 'Developer', 'Software Developer', 'Software Engineer', 'Kỹ sư phần mềm', 'Frontend Developer', 'Backend Developer', 'Fullstack Developer', 'Web Developer', 'Mobile Developer', 'Senior Developer', 'Senior Engineer', 'Lead Developer', 'Junior Developer', 'Junior Engineer', 'Fresher IT', 'Intern IT', 'Thực tập sinh IT', 'Entry Level', 'Mid Level', 'Remote IT', 'Hybrid IT', 'Onsite', 'Offshore', 'Nearshore', 'Outsourcing', 'ODC', 'Python', 'Java', 'JavaScript', 'TypeScript', 'PHP', '.NET', 'C#', 'Golang', 'Go', 'Ruby', 'Kotlin', 'Swift', 'C', 'C++', 'Rust', 'Scala', 'Objective-C', 'Shell Script', 'Bash', 'PowerShell', 'SQL', 'NoSQL', 'Database', 'DB', 'RDBMS', 'MySQL', 'PostgreSQL', 'Oracle', 'SQL Server', 'MongoDB', 'Redis',

In [20]:
# Cell 4: Load job đã xử lý trước đó (tránh duplicate)
def load_existing_summaries():
    if not os.path.exists(SUMMARY_FILE):
        return set()
    existing_ids = set()
    with open(SUMMARY_FILE, "r", encoding="utf-8") as f:
        for i, line in enumerate(f):
            if (i + 1) % 500 == 0:
                print(f"   Loading summaries cũ: {i+1} dòng...")
            if line.strip():
                try:
                    entry = json.loads(line)
                    existing_ids.add(entry['job_id'])
                except:
                    pass
    print(f"Đã load {len(existing_ids)} job đã xử lý trước đó.")
    return existing_ids

processed_job_ids = load_existing_summaries()
seen_job_ids = processed_job_ids.copy()

   Loading summaries cũ: 500 dòng...
   Loading summaries cũ: 1000 dòng...
   Loading summaries cũ: 1500 dòng...
   Loading summaries cũ: 2000 dòng...
   Loading summaries cũ: 2500 dòng...
   Loading summaries cũ: 3000 dòng...
   Loading summaries cũ: 3500 dòng...
   Loading summaries cũ: 4000 dòng...
   Loading summaries cũ: 4500 dòng...
   Loading summaries cũ: 5000 dòng...
   Loading summaries cũ: 5500 dòng...
   Loading summaries cũ: 6000 dòng...
   Loading summaries cũ: 6500 dòng...
   Loading summaries cũ: 7000 dòng...
   Loading summaries cũ: 7500 dòng...
   Loading summaries cũ: 8000 dòng...
   Loading summaries cũ: 8500 dòng...
   Loading summaries cũ: 9000 dòng...
   Loading summaries cũ: 9500 dòng...
   Loading summaries cũ: 10000 dòng...
Đã load 10093 job đã xử lý trước đó.


In [21]:
# Cell 5: Hàm fetch danh sách job (pagination)
base_url = "https://www.linkedin.com/voyager/api/voyagerJobsDashJobCards"

def fetch_job_list(keyword, seen_job_ids):
    query_string_base = (
        f"decorationId=com.linkedin.voyager.dash.deco.jobs.search.JobSearchCardsCollectionLite-88"
        f"&count={COUNT}"
        f"&q=jobSearch"
        f"&query=(origin:SWITCH_SEARCH_VERTICAL,keywords:{keyword},spellCorrectionEnabled:true)"
        f"&servedEventEnabled=false"
    )

    new_jobs = []
    start = 0
    total = None
    page = 1

    print(f"[{keyword.upper()}] Bắt đầu tìm kiếm...")

    while total is None or start < total:
        print(f"   Trang {page} (start={start})...", end=" ")
        url = f"{base_url}?{query_string_base}&start={start}"
        try:
            response = session.get(url, timeout=30)
            if response.status_code != 200:
                print(f"HTTP {response.status_code}")
                break
            data = response.json()
        except Exception as e:
            print(f"Lỗi: {e}")
            break

        included = data.get('included', [])
        if total is None:
            total = data.get('data', {}).get('paging', {}).get('total', 0)
            print(f"→ Tổng: {total}")

        jobs_found = 0
        for item in included:
            if (item.get('$type') == 'com.linkedin.voyager.dash.jobs.JobPosting' and
                item.get('entityUrn', '').startswith('urn:li:fsd_jobPosting:')):
                job_id = item['entityUrn'].split(':')[-1]
                if job_id not in seen_job_ids:
                    seen_job_ids.add(job_id)
                    new_jobs.append(item)
                    jobs_found += 1

        print(f"+{jobs_found} job mới")
        if jobs_found == 0 and start > 0:
            break

        start += COUNT
        page += 1
        time.sleep(1)

    print(f"[{keyword.upper()}] Hoàn thành: +{len(new_jobs)} job mới\n")
    return new_jobs

In [22]:
# Cell 6: Hàm fetch chi tiết job (có retry)
def fetch_job_detail(job_id):
    url_temp = "https://www.linkedin.com/voyager/api/graphql"
    urn_raw = f"urn:li:fsd_jobPosting:{job_id}"
    urn_encoded = quote(urn_raw, safe="")
    variables = f"(jobPostingUrn:{urn_encoded})"
    query_id = "voyagerJobsDashJobPostings.891aed7916d7453a37e4bbf5f1f60de4"  # Có thể thay đổi theo thời gian
    url = f"{url_temp}?variables={variables}&queryId={query_id}"

    for attempt in range(RETRY_COUNT):
        try:
            response = session.get(url, timeout=20)
            if response.status_code != 200:
                if attempt == RETRY_COUNT - 1:
                    return None, None
                time.sleep(1)
                continue
            data = response.json()
            included = data.get('included', [])
            for item in included:
                if item.get('$type') == 'com.linkedin.voyager.dash.jobs.JobPosting':
                    return item, included
            return None, None
        except Exception:
            if attempt == RETRY_COUNT - 1:
                return None, None
            time.sleep(2 ** attempt)
    return None, None

In [23]:
# Cell 7: Hàm extract thông tin job
def extract_job_info(job_detail, included=None):
    if not job_detail:
        return None

    def resolve_urn(urn, target_type):
        if not urn or not included:
            return None
        for item in included:
            if item.get('entityUrn') == urn and item.get('$type') == target_type:
                return item
        return None

    job_id = job_detail.get('entityUrn', '').split(':')[-1]
    job_title = job_detail.get('title', '')
    company_name = job_detail.get('companyDetails', {}).get('name')

    location = None
    loc_urn = job_detail.get('*location')
    if loc_urn and isinstance(loc_urn, str):
        geo = resolve_urn(loc_urn, 'com.linkedin.voyager.dash.common.Geo')
        if geo:
            location = geo.get('defaultLocalizedName') or geo.get('abbreviatedLocalizedName')
    if not location:
        location = job_detail.get('formattedLocation')

    description = job_detail.get('description', {}).get('text', '')

    posting_date = None
    if job_detail.get('listedAt'):
        try:
            posting_date = datetime.datetime.fromtimestamp(job_detail['listedAt'] / 1000).strftime('%Y-%m-%d')
        except:
            pass

    # Các field khác tương tự...
    job_url = f"https://www.linkedin.com/jobs/view/{job_id}"

    return {
        'job_id': job_id,
        'job_title': job_title,
        'company': company_name,
        'location': location,
        'description': description,
        'posting_date': posting_date,
        'job_url': job_url,
        'crawled_at': datetime.datetime.now().isoformat(),
        # Thêm các field khác nếu cần
    }

In [24]:
# Cell 8: CHẠY SCRAPING CHÍNH (chạy cell này cuối cùng)

all_new_jobs = []

for idx, kw in enumerate(keywords_list, 1):
    print(f"[{idx}/{len(keywords_list)}] Từ khóa: '{kw}'")
    new_jobs = fetch_job_list(kw, seen_job_ids)
    all_new_jobs.extend(new_jobs)

if not all_new_jobs:
    print("Không có job mới nào. Kết thúc.")
else:
    print(f"\nTổng {len(all_new_jobs)} job mới cần lấy detail.\n")

    file_exists = os.path.exists(CSV_FILE)
    with open(CSV_FILE, 'a', newline='', encoding='utf-8-sig') as csvfile:
        fieldnames = ['job_id', 'job_title', 'company', 'location', 'description',
                      'posting_date', 'job_url', 'crawled_at']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        if not file_exists or os.stat(CSV_FILE).st_size == 0:
            writer.writeheader()

        success = 0
        failed = 0

        with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
            futures = {executor.submit(fetch_job_detail, job['entityUrn'].split(':')[-1]): job for job in all_new_jobs}

            processed = 0
            total = len(futures)
            for future in as_completed(futures):
                processed += 1
                job = futures[future]
                job_id = job['entityUrn'].split(':')[-1]
                title = job.get('title', 'No title')

                job_detail, included = future.result()
                source = job_detail or job
                info = extract_job_info(source, included=included)

                if info:
                    writer.writerow(info)
                    csvfile.flush()

                    # Lưu summary
                    entry = {
                        "job_id": job_id,
                        "title": job.get('title'),
                        "entityUrn": job['entityUrn'],
                        "crawled_at": datetime.datetime.now().isoformat(),
                    }
                    with open(SUMMARY_FILE, "a", encoding="utf-8") as f:
                        f.write(json.dumps(entry, ensure_ascii=False) + "\n")

                    success += 1
                    print(f"[{processed}/{total}] [OK] {job_id} - {title}")
                else:
                    failed += 1
                    print(f"[{processed}/{total}] [FAIL] {job_id} - {title}")

                time.sleep(0.25)

    print("\nHOÀN THÀNH!")
    print(f"Thành công: {success} | Fail: {failed} | File: {CSV_FILE}")

[1/229] Từ khóa: 'IT'
[IT] Bắt đầu tìm kiếm...
   Trang 1 (start=0)... 

→ Tổng: 795
+0 job mới
   Trang 2 (start=100)... +0 job mới
[IT] Hoàn thành: +0 job mới

[2/229] Từ khóa: 'Information Technology'
[INFORMATION TECHNOLOGY] Bắt đầu tìm kiếm...
   Trang 1 (start=0)... → Tổng: 164
+0 job mới
   Trang 2 (start=100)... +1 job mới
[INFORMATION TECHNOLOGY] Hoàn thành: +1 job mới

[3/229] Từ khóa: 'Công nghệ thông tin'
[CÔNG NGHỆ THÔNG TIN] Bắt đầu tìm kiếm...
   Trang 1 (start=0)... → Tổng: 2413
+0 job mới
   Trang 2 (start=100)... +0 job mới
[CÔNG NGHỆ THÔNG TIN] Hoàn thành: +0 job mới

[4/229] Từ khóa: 'CNTT'
[CNTT] Bắt đầu tìm kiếm...
   Trang 1 (start=0)... → Tổng: 19
+0 job mới
[CNTT] Hoàn thành: +0 job mới

[5/229] Từ khóa: 'Computer Science'
[COMPUTER SCIENCE] Bắt đầu tìm kiếm...
   Trang 1 (start=0)... → Tổng: 3180
+0 job mới
   Trang 2 (start=100)... +0 job mới
[COMPUTER SCIENCE] Hoàn thành: +0 job mới

[6/229] Từ khóa: 'CS'
[CS] Bắt đầu tìm kiếm...
   Trang 1 (start=0)... → Tổng: 2171
+0 job mới
   Trang 2 (start=100)... +0 job mới
[CS] Hoàn thành: